In [4]:
%load_ext autoreload
%autoreload 2
import matplotlib.pyplot as plt
import datajoint as dj
import spyglass.common as sgc
import spyglass.position.v1 as sgp

from spyglass.utils.nwb_helper_fn import get_nwb_copy_filename
from spyglass.position import PositionOutput
dj.config["filepath_checksum_size_limit"] = 1 * 1024**3
dj.config.save_global()
import spyglass
import os

In [2]:
(sgp.TrodesPosParams() & {'trodes_pos_params_name': 'single_led_upsampled'}).fetch('params')

array([{'max_separation': 10000.0, 'max_speed': 300.0, 'position_smoothing_duration': 0.125, 'speed_smoothing_std_dev': 0.1, 'orient_smoothing_std_dev': 0.001, 'led1_is_front': 1, 'is_upsampled': 1, 'upsampling_sampling_rate': 500, 'upsampling_interpolation_method': 'linear'}],
      dtype=object)

## Define Animal Sessions

In [5]:
import os
# nwb_file_names = [
#     #Olive
#     "Olive20220624_.nwb",
#     "Olive20220627_.nwb",
#     "Olive20220628_.nwb",
#     "Olive20220629_.nwb",
#     "Olive20220630_.nwb",
#     "Olive20220701_.nwb",
#     "Olive20220702_.nwb",
#     "Olive20220705_.nwb",
#     "Olive20220706_.nwb",
#     #Yoshi
#     "Yoshi20220509_.nwb",
#     "Yoshi20220510_.nwb", #permissions
#     "Yoshi20220511_.nwb",
#     "Yoshi20220519_.nwb",
#     "Yoshi20220525_.nwb",
#     #Winnie
#     "Winnie20220712_.nwb",
#     "Winnie20220713_.nwb",
#     "Winnie20220714_.nwb",
#     "Winnie20220715_.nwb",
#     "Winnie20220716_.nwb",
#     #Wallie
#     "Wallie20220911_.nwb",
#     "Wallie20220912_.nwb",
#     "Wallie20220913_.nwb",
#     "Wallie20220916_.nwb",
#     #Totoro
#     "Totoro20220524_.nwb",
#     "Totoro20220525_.nwb",
#     "Totoro20220526_.nwb",
#     "Totoro20220527_.nwb",
#     "Totoro20220530_.nwb",
#     "Totoro20220531_.nwb",
#     "Totoro20220602_.nwb", #permissions
#     "Totoro20220603_.nwb",
#     "Totoro20220606_.nwb",
#     "Totoro20220621_.nwb",
#     #Odins
#     # "Odins20220107_.nwb",
#     # "Odins20220110_.nwb",
#     # "Odins20220111_.nwb",
#     # "Odins20220113_.nwb",
#     # "Odins20220118_.nwb",
#     # "Odins20220119_.nwb",
#     # "Odins20220123_.nwb",
#     # "Odins20220124_.nwb",
#     # "Odins20220126_.nwb", ???
#     #Banner
#     "Banner20211221_.nwb",
#     "Banner20211222_.nwb",
#     "Banner20211227_.nwb",
#     "Banner20211228_.nwb", #permissions
#     "Banner20211230_.nwb",
#     "Banner20211231_.nwb",
#     "Banner20220103_.nwb",
#     "Banner20220105_.nwb",
#     "Banner20220123_.nwb",
#     "Banner20220124_.nwb",
#     "Banner20220126_.nwb",
# ]

# nwb_file_names.extend([
#     #Olive
#     "Olive20220707_.nwb",
#     "Olive20220708_.nwb",
#     "Olive20220711_.nwb",
#     #Yoshi
#     "Yoshi20220513_.nwb",
#     "Yoshi20220516_.nwb",
#     "Yoshi20220517_.nwb",
#     "Yoshi20220518_.nwb",
#     #Winnie
#     "Winnie20220717_.nwb",
#     "Winnie20220718_.nwb",
#     "Winnie20220719_.nwb",
#     "Winnie20220720_.nwb",
#     "Winnie20220722_.nwb",
#     #Wallie
#     "Wallie20220914_.nwb",
#     "Wallie20220915_.nwb",
#     "Wallie20220922_.nwb",
#     #Totoro
#     "Totoro20220607_.nwb",
#     "Totoro20220610_.nwb",
#     "Totoro20220613_.nwb",
#     "Totoro20220614_.nwb",
#     "Totoro20220615_.nwb",
#     "Totoro20220616_.nwb",
#     "Totoro20220617_.nwb",
#     #Odins
#     "Odins20220226_.nwb",
#     #Banner
#     "Banner20220215_.nwb",
#     "Banner20220216_.nwb",
#     "Banner20220223_.nwb",
#     "Banner20220224_.nwb",
#     "Banner20220225_.nwb",
# ])
    
# np.isnan(np.mean([]))
animal_names = ["Banner","Danvers","Odins","Totoro","Wallie","Winnie","Yoshi","Olive","Frodo","Bilbo"]
# animal_names = ["Frodo", "Winnie","Bilbo"]
from spyglass.common import Session
nwb_file_names = []
[nwb_file_names.extend((Session() & {"subject_id":animal}).fetch("nwb_file_name")) for animal in animal_names]
# nwb_file_names.pop(nwb_file_names.index('samtest20230817_.nwb'))
# nwb_file_names.pop(nwb_file_names.index('samtestb20230817_.nwb'))

# nwb_file_names = ["Frodo20230808_.nwb"]

#write permission?
analysis_path = "/stelmo/nwb/analysis/"
for nwb_file in nwb_file_names:
    if not os.path.isdir(analysis_path+nwb_file.split('_')[0]):
        print(f'No analysis yet for: {nwb_file}')
    elif not os.access(analysis_path+nwb_file.split('_')[0],os.W_OK):
        print(f'No write permissions for: {nwb_file}nwb_file')


## Populate all position tables

In [17]:
import warnings
warnings.filterwarnings('ignore',category=UserWarning)

# nwb_file_names = [ 'Totoro20220530_.nwb']
trodes_pos_params_name = "single_led"
trodes_pos_params_name = "single_led_upsampled" #use this for decoding

# nwb_file_names = ["SB2spikegadgets20220224_.nwb"]
for nwb_file_name in nwb_file_names:
    # if os.path.isdir(analysis_path+nwb_file_name.split('_')[0]) and not os.access(analysis_path+nwb_file_name.split('_')[0],os.W_OK):
    #     print(f'No write permissions for: {nwb_file}nwb_file')
    #     continue
    if nwb_file_name in ['samtestb20230817_.nwb','SB2spikegadgets20220224_.nwb']:
        print(f'No write permissions for: {nwb_file}nwb_file')
        continue
    
    nwb_copy_file_name = nwb_file_name#get_nwb_copy_filename(nwb_file_name)
    #check that raw position info exists
    if len(sgc.RawPosition() & {"nwb_file_name":nwb_copy_file_name})==0:
        print(f"{nwb_file_name} doesn't have raw position data")
    

    #loop through position intervals for this session
    for interval_list_name in (sgc.RawPosition()&{"nwb_file_name": nwb_file_name,}).fetch("interval_list_name"):
        start_key = {"nwb_file_name": nwb_copy_file_name,"interval_list_name": interval_list_name, "trodes_pos_params_name": trodes_pos_params_name}
        if sgp.TrodesPosV1 & start_key:
            continue
        # sgp.TrodesPosSelection() & {**start_key, "trodes_pos_params_name": trodes_pos_params_name,})
        sgp.TrodesPosSelection.insert1({**start_key,}, skip_duplicates=True,)
        
        trodes_key =(sgp.TrodesPosSelection() &  {**start_key,}).fetch1("KEY")
        print(trodes_key)
        #skip ones that are already populated
        if len(sgp.TrodesPosV1()&trodes_key)>0:
            continue
        # sgp.TrodesPosV1.populate(trodes_key)
        # print(PositionOutput()&trodes_key)

sgp.TrodesPosV1.populate(processes=8)     



{'nwb_file_name': 'Banner20211221_.nwb', 'interval_list_name': 'pos 0 valid times', 'trodes_pos_params_name': 'single_led_upsampled'}
{'nwb_file_name': 'Banner20211221_.nwb', 'interval_list_name': 'pos 1 valid times', 'trodes_pos_params_name': 'single_led_upsampled'}
{'nwb_file_name': 'Banner20211222_.nwb', 'interval_list_name': 'pos 0 valid times', 'trodes_pos_params_name': 'single_led_upsampled'}
{'nwb_file_name': 'Banner20211222_.nwb', 'interval_list_name': 'pos 1 valid times', 'trodes_pos_params_name': 'single_led_upsampled'}
{'nwb_file_name': 'Banner20211222_.nwb', 'interval_list_name': 'pos 2 valid times', 'trodes_pos_params_name': 'single_led_upsampled'}
{'nwb_file_name': 'Banner20211222_.nwb', 'interval_list_name': 'pos 3 valid times', 'trodes_pos_params_name': 'single_led_upsampled'}
{'nwb_file_name': 'Banner20211222_.nwb', 'interval_list_name': 'pos 4 valid times', 'trodes_pos_params_name': 'single_led_upsampled'}
{'nwb_file_name': 'Banner20211227_.nwb', 'interval_list_name':

In [28]:
for animal in animal_names:
    key_list = sgp.TrodesPosSelection() & f"(nwb_file_name LIKE '{animal}%')" & {'trodes_pos_params_name': 'single_led_upsampled'}
    sgp.TrodesPosV1.populate(key_list,
                              processes=16,
                              display_progress=True) 

In [14]:
sgp.TrodesPosSelection() & {'trodes_pos_params_name': 'single_led_upsampled'}

nwb_file_name name of the NWB file,interval_list_name descriptive name of this interval list,trodes_pos_params_name name for this set of parameters
minirec20230622_syt2_.nwb,pos 0 valid times,single_led_upsampled
samtestb20230817_.nwb,pos 0 valid times,single_led_upsampled
samtestb20230817_.nwb,pos 1 valid times,single_led_upsampled
samtestb20230817_.nwb,pos 2 valid times,single_led_upsampled
samtestb20230817_.nwb,pos 3 valid times,single_led_upsampled
samtestb20230817_.nwb,pos 4 valid times,single_led_upsampled
samtestb20230817_.nwb,pos 5 valid times,single_led_upsampled
samtestb20230817_.nwb,pos 6 valid times,single_led_upsampled
samtestb20230817_.nwb,pos 7 valid times,single_led_upsampled
SB2spikegadgets20220224_.nwb,pos 0 valid times,single_led_upsampled


In [31]:
# [x for x in sgp.TrodesPosV1().fetch("nwb_file_name")if "Frodo" in x]#& trodes_key
print(start_key)
(sgc.RawPosition()&{"nwb_file_name": nwb_file_name}&start_key).fetch1_dataframe()
# sgp.TrodesPosV1()&{"nwb_file_name": nwb_file_name}
# nwb_file_name

[2023-10-18 14:08:25,658][WARNING]: Skipped checksum for file with hash: 0486d4af-f2ca-b3c2-a908-5e322fe471a7, and path: /stelmo/nwb/raw/Winnie20220712_.nwb


{'nwb_file_name': 'Winnie20220712_.nwb', 'interval_list_name': 'pos 1 valid times'}


,video_frame_ind,non_repeat_timestamp_labels,xloc,yloc,xloc2,yloc2
time,,,,,,
1.657659e+09,8,1,172,950,0,0
1.657659e+09,9,1,172,950,0,0
1.657659e+09,10,1,172,950,0,0
1.657659e+09,11,1,172,950,0,0
1.657659e+09,12,1,172,950,0,0
...,...,...,...,...,...,...
1.657660e+09,40012,1,188,928,0,0
1.657660e+09,40013,1,189,929,0,0
1.657660e+09,40014,1,191,929,0,0


In [25]:
from pynwb import NWBHDF5IO

file = f"/stelmo/nwb/raw/{nwb_file_name}"
with NWBHDF5IO(file,'r') as io:
    nwbfile = io.read()
    print(nwbfile.processing['behavior']['position'])

position pynwb.behavior.Position at 0x139847447277472
Fields:
  spatial_series: {
    led_0_series_1 <class 'pynwb.behavior.SpatialSeries'>,
    led_0_series_10 <class 'pynwb.behavior.SpatialSeries'>,
    led_0_series_12 <class 'pynwb.behavior.SpatialSeries'>,
    led_0_series_13 <class 'pynwb.behavior.SpatialSeries'>,
    led_0_series_2 <class 'pynwb.behavior.SpatialSeries'>,
    led_0_series_3 <class 'pynwb.behavior.SpatialSeries'>,
    led_0_series_4 <class 'pynwb.behavior.SpatialSeries'>,
    led_0_series_5 <class 'pynwb.behavior.SpatialSeries'>,
    led_0_series_6 <class 'pynwb.behavior.SpatialSeries'>,
    led_0_series_7 <class 'pynwb.behavior.SpatialSeries'>,
    led_0_series_8 <class 'pynwb.behavior.SpatialSeries'>,
    led_0_series_9 <class 'pynwb.behavior.SpatialSeries'>,
    led_1_series_1 <class 'pynwb.behavior.SpatialSeries'>,
    led_1_series_10 <class 'pynwb.behavior.SpatialSeries'>,
    led_1_series_12 <class 'pynwb.behavior.SpatialSeries'>,
    led_1_series_13 <class '

In [18]:
# print(start_key)
# sgp.TrodesPosSelection() & {**start_key, "trodes_pos_params_name": trodes_pos_params_name,}
PositionOutput().TrodesPosV1()#&{'nwb_file_name':nwb_copy_file_name}#trodes_key
# position_intervals
# sgc.IntervalList() &{"nwb_file_name":nwb_copy_file_name}
sgp.TrodesPosParams()
# sgp.TrodesPosV1.__dict__

mappingproxy({'__module__': 'spyglass.position.v1.position_trodes_position',
              '__doc__': '\n    Table to calculate the position based on Trodes tracking\n    \nTable definition:\n\n\n    -> TrodesPosSelection\n    ---\n    -> AnalysisNwbfile\n    position_object_id : varchar(80)\n    orientation_object_id : varchar(80)\n    velocity_object_id : varchar(80)\n    ',
              'definition': '\n    -> TrodesPosSelection\n    ---\n    -> AnalysisNwbfile\n    position_object_id : varchar(80)\n    orientation_object_id : varchar(80)\n    velocity_object_id : varchar(80)\n    ',
              'make': <function spyglass.position.v1.position_trodes_position.TrodesPosV1.make(self, key)>,
              'calculate_position_info_from_spatial_series': <staticmethod at 0x7f7463c90c10>,
              'fetch_nwb': <function spyglass.position.v1.position_trodes_position.TrodesPosV1.fetch_nwb(self, *attrs, **kwargs)>,
              'fetch1_dataframe': <function spyglass.position.v1.positi

In [7]:
position_intervalse = [name for name in (sgc.IntervalList() &{"nwb_file_name":nwb_copy_file_name}).fetch('interval_list_name') if 'pos' in name]

In [5]:
sgc.Session() & {"subject_id":"Frodo"}

nwb_file_name name of the NWB file,subject_id,institution_name,lab_name,session_id,session_description,session_start_time,timestamps_reference_time,experiment_description
Frodo20230809_.nwb,Frodo,"University of California, San Francisco",Loren Frank Lab,2,Medial Septal stimulation,2023-08-09 09:44:25,1970-01-01 00:00:00,Medial Septal rhythmic and theta phase specific stimulation
Frodo20230810_.nwb,Frodo,"University of California, San Francisco",Loren Frank Lab,3,Medial Septal stimulation,2023-08-10 09:50:21,1970-01-01 00:00:00,Medial Septal rhythmic and theta phase specific stimulation
Frodo20230814_.nwb,Frodo,"University of California, San Francisco",Loren Frank Lab,5,Medial Septal stimulation,2023-08-14 09:21:36,1970-01-01 00:00:00,Medial Septal rhythmic and theta phase specific stimulation
Frodo20230815_.nwb,Frodo,"University of California, San Francisco",Loren Frank Lab,6,Medial Septal stimulation,2023-08-15 09:25:35,1970-01-01 00:00:00,Medial Septal rhythmic and theta phase specific stimulation
Frodo20230816_.nwb,Frodo,"University of California, San Francisco",Loren Frank Lab,7,Medial Septal stimulation,2023-08-16 10:21:13,1970-01-01 00:00:00,Medial Septal rhythmic and theta phase specific stimulation


In [4]:
sgc.Session() & {"nwb_file_name":"Winnie20220714_.nwb"}

nwb_file_name name of the NWB file,subject_id,institution_name,lab_name,session_id,session_description,session_start_time,timestamps_reference_time,experiment_description
